In [168]:
sharing_factor = 20

In [34]:
import math
import numpy
import random

In [172]:
def fitness(indiv) -> tuple:
  return indiv

In [187]:
# Cette cellule  permet de faire un combat entre deux individus sur un population donné et de soit renvoyer le gagnant (True,_) soit déclarer un match nul (False,0)


# Fonction qui permet de savoir si un individu en domine un autre

# Prend en entrée la fitness des deux individus
# Renvoie en sortie False si l'individu 2 domine l'individu 1. Renvoie True sinon

def dominance_check(fit1,fit2) -> bool:
  if (fit1[0] >= fit2[0] and fit1[1] > fit2[1]) or (fit1[0] > fit2[0] and fit1[1] >= fit2[1]):
    return False
  else:
    return True


# Fonction qui effectue les matchs

# Prend en entrée les deux individus et la sous population sur laquelle ils vont se battre
# Renvoie (False,0) si les individus sont dominés ou domine toute la sous population
# Renvoie (True,_) si un individu est gagnant, "_" étant cet individu

def match_tournois(indiv1,indiv2,population_fit) -> tuple :
  fit1 = fitness(indiv1)
  fit2 = fitness(indiv2)

  res_indiv1 = True
  res_indiv2 = True
  i = 0
  
  while (res_indiv1 or res_indiv2) and i < len(population_fit) :
    if res_indiv1:
      res_indiv1 = dominance_check(fit1,population_fit[i])
    
    if res_indiv2:
      res_indiv2 = dominance_check(fit2,population_fit[i])

    i += 1 

  if not (res_indiv1 ^ res_indiv2):
    return (False,0)
  
  if res_indiv1:
    return (True, indiv1)

  else:
    return (True, indiv2)

In [134]:
'''
indiv1 = (1,2)
indiv2 = (0,1)
population = [
    (1,3),
    (4,2),
    (1,2),
    (3,3),
    (1,5),
    (0,1)
]


res = match_tournois(indiv1,indiv2, population)

print(res)

# res doit etre égale à (True, (0, 1))
'''

'\nindiv1 = (1,2)\nindiv2 = (0,1)\npopulation = [\n    (1,3),\n    (4,2),\n    (1,2),\n    (3,3),\n    (1,5),\n    (0,1)\n]\n\n\nres = match_tournois(indiv1,indiv2, population)\n\nprint(res)\n\n# res doit etre égale à (True, (0, 1))\n'

In [176]:
# Fonction qui permet de calculer la "shared fitness" d'un individu par rapport à une population


# Fonction à vectoriser pour calculer le niche count d'un individu

def sharing_function(indiv1, indiv2):
  distance = math.sqrt((indiv1[0]-indiv2[0])*(indiv1[0]-indiv2[0])+(indiv1[1]-indiv2[1])*(indiv1[1]-indiv2[1]))
  sh = 1 - (distance/sharing_factor)
  if sh < 0:
    sh = 0
  return sh


# Prend en entrée la fitness d'un individu et la fitness de la population
# Renvoie en sortie la "shared fitness" de l'individu

def sharing_procedure(indiv, population_fit) -> float:
  
  fit = indiv

  vectorized_sharing = numpy.vectorize(sharing_function, signature ='(n),(m)->()')
  
  niche_count = sum(vectorized_sharing(fit,population_fit))
  return numpy.mean(fit)/niche_count

In [165]:
'''
individual = (1,2)
population = [
    (1,3),
    (4,2)
]


res = sharing_procedure(individual, population)

print(res)

# Res doit etre égale à 1.2499999999999998
'''

'\nindividual = (1,2)\npopulation = [\n    (1,3),\n    (4,2)\n]\n\n\nres = sharing_procedure(individual, population)\n\nprint(res)\n\n# Res doit etre égale à 1.2499999999999998\n'

In [189]:
def tournois(population,winner_length)->list:
  winners = []
  while len(winners) < winner_length:
    indiv1 = random.choice(population)
    indiv2 = random.choice(population)

    while indiv1 == indiv2 :
      indiv2 = random.choice(population)

    #vectorized_fit = numpy.vectorize(fitness,signature= '(m)->(l)')
    population = list(map(fitness, population))
    sub_pop_len = random.randint(1,len(population))
    sub_pop = random.choices(population,k=sub_pop_len)


    (res,winner) = match_tournois(indiv1,indiv2,sub_pop)
    if res:
      winners.append((winner,0))
    else:
      sf = sharing_procedure(indiv1,population)
      winners.append((indiv1,sf))

      sf = sharing_procedure(indiv2,population)
      winners.append((indiv2,sf))

  return winners

In [190]:
population = [
    (1,3),
    (4,2),
    (1,2),
    (3,3),
    (1,5),
    (0,1)
]

res = tournois(population,3)

print(res)

[((1, 5), 0.5780022373201288), ((1, 3), 0.3649579947740944), ((4, 2), 0)]
